# Dialogue and Narrative Coursework - Subtask 1

In [1]:
## imports
# %%capture
# !pip install datasets

from datasets import load_dataset, Dataset
import pandas as pd

In [2]:
# # Link Google Drive to get new dataset
# from google.colab import drive
# drive.mount('/gdrive')

# # %cd drive/MyDrive/Colab\ Notebooks/Dialogue\ &\ Narrative
# %cd ~
# %cd /gdrive/My\ Drive
# %cd Colab\ Notebooks/Dialogue\ &\ Narrative
# print()

# %ls .

## Load the data

In [3]:
## A function to load a specific doc2dial dataset

def load_doc2dial_dataset(name='dialogue_domain', split='train'):
  cache_dir = "./data_cache"

  return load_dataset(
      "doc2dial",
      name=name,
      split=split,
      ignore_verifications=True,
      cache_dir=cache_dir,
  )


## Understanding an example

In [4]:
## Load documents 

documents = load_doc2dial_dataset(
              name="document_domain",
              split="train",
            )

Reusing dataset doc2dial (./data_cache/doc2dial/document_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee)


In [5]:
print(documents[0].keys())
documents[0]['spans'][0]

dict_keys(['doc_html_raw', 'doc_html_ts', 'doc_id', 'doc_text', 'domain', 'spans', 'title'])


{'end_sec': 61,
 'end_sp': 61,
 'id_sec': 't_0',
 'id_sp': '1',
 'parent_titles': '[]',
 'start_sec': 0,
 'start_sp': 0,
 'tag': 'h2',
 'text_sec': '\n\nBenefits Planner: Survivors | Planning For Your Survivors \n',
 'text_sp': '\n\nBenefits Planner: Survivors | Planning For Your Survivors \n',
 'title': 'Benefits Planner: Survivors | Planning For Your Survivors'}

In [6]:
## Load dialogue turns

dialogues = load_doc2dial_dataset(
              name="dialogue_domain",
              split="validation",
            )

Reusing dataset doc2dial (./data_cache/doc2dial/dialogue_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee)


In [7]:
print(dialogues[0].keys())
dialogues[0]['domain']

dict_keys(['dial_id', 'doc_id', 'domain', 'turns'])


'dmv'

For subtask 1, the gound truths are specified in the reading comprehension dataset 'doc2dial_rc'. Let's show this on the first example from the validation set.

In [8]:
## Using the doc2dial_rc dataset (also used for evaluation)

qas = load_doc2dial_dataset(
              name="doc2dial_rc",
              split="validation",
            )

Reusing dataset doc2dial (./data_cache/doc2dial/doc2dial_rc/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee)


In [9]:
## The following display will be used as ground truth for evaluation
ref = {'id':qas[0]['id'], 'answers':qas[0]['answers']}
print(ref)
print(qas)
qas[0]

{'id': 'dea7174409afbfe0af0ace21e7f318ae_1', 'answers': {'answer_start': [1796], 'text': ['Because we all pay indirectly for crashes involving uninsured motorists , New York State requires every motorist to maintain auto insurance every single day a vehicle is registered. DMV works with insurance companies to electronically monitor your insurance coverage , ']}}
Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'domain'],
    num_rows: 4002
})


{'answers': {'answer_start': [1796],
  'text': ['Because we all pay indirectly for crashes involving uninsured motorists , New York State requires every motorist to maintain auto insurance every single day a vehicle is registered. DMV works with insurance companies to electronically monitor your insurance coverage , ']},
 'context': 'Many DMV customers make easily avoidable mistakes that cause them significant problems, including encounters with law enforcement and impounded vehicles. Because we see customers make these mistakes over and over again , we are issuing this list of the top five DMV mistakes and how to avoid them. \n\n1. Forgetting to Update Address \nBy statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. It is not sufficient to only: write your new address on the back of your old license; tell the

The answer corresponding to question `dea7174409afbfe0af0ace21e7f318ae_1` most likely has id `dea7174409afbfe0af0ace21e7f318ae_2`. So let's check it using the given dialogue and document domains datasets. We should also check that this answer corresponds to a span starting at `1796` in the associated document.

In [10]:
## Let's find anwer 'dea7174409afbfe0af0ace21e7f318ae_2'

question_id = 'dea7174409afbfe0af0ace21e7f318ae_2'
dial = 'dea7174409afbfe0af0ace21e7f318ae'
turn = 2

for ex in dialogues:
  if ex['dial_id'] == dial:

    doc_id = ex['doc_id']   ## corresponding document id
    span_ids = []           ## corresponding span ids

    for tr in ex['turns']:
      if tr['turn_id'] == turn:
        print(tr)

        for sp in tr['references']:
          span_ids.append(sp['sp_id'])

{'da': 'respond_solution', 'references': [{'label': 'solution', 'sp_id': '24'}, {'label': 'solution', 'sp_id': '25'}, {'label': 'solution', 'sp_id': '26'}], 'role': 'agent', 'turn_id': 2, 'utterance': 'You will need to get insurance or we will suspend your registration and license'}


In [11]:
## Let's get the correpond answer in the document_domain

for doc in documents:
  if doc['doc_id'] == doc_id:

    ans = ''
    for span in doc['spans']:
      if span['id_sp'] in span_ids:
        ans += span['text_sp']

print(ans)

Because we all pay indirectly for crashes involving uninsured motorists , New York State requires every motorist to maintain auto insurance every single day a vehicle is registered. DMV works with insurance companies to electronically monitor your insurance coverage , 


In [12]:
## Let's check that the effectively starts at 1796

for doc in documents:
  if doc['doc_id'] == doc_id:
    for span in doc['spans']:
      if span['start_sp'] == 1796:
        print(span['text_sp'])

Because we all pay indirectly for crashes involving uninsured motorists , 
Because we all pay indirectly for crashes involving uninsured motorists , 
Because we all pay indirectly for crashes involving uninsured motorists , 
Because we all pay indirectly for crashes involving uninsured motorists , 
Because we all pay indirectly for crashes involving uninsured motorists , 
Because we all pay indirectly for crashes involving uninsured motorists , 
Because we all pay indirectly for crashes involving uninsured motorists , 


It checks out! Now that we know that our ground truths are essentially doc2dial_rc, let see what our predictions should look like in order to comply with the evaluation script.

In [13]:
## Examples of predictions
[
    {
        "id": "dea7174409afbfe0af0ace21e7f318ae_1",
        "prediction_text": "Because we all pay indirectly for crashes involving uninsured motorists , New York State requires every motorist to maintain auto insurance every single day a vehicle is registered. DMV works with insurance companies to electronically monitor your insurance coverage ,",
        "no_answer_probability": 0.883711576461792
    },
    {
        "id": "dea7174409afbfe0af0ace21e7f318ae_9",
        "prediction_text": "we mail you an insurance inquiry letter to allow you to clear up the problem. We send 500,000 inquiry letters a year. ",
        "no_answer_probability": 0.0004379140445962548
    }
]

[{'id': 'dea7174409afbfe0af0ace21e7f318ae_1',
  'prediction_text': 'Because we all pay indirectly for crashes involving uninsured motorists , New York State requires every motorist to maintain auto insurance every single day a vehicle is registered. DMV works with insurance companies to electronically monitor your insurance coverage ,',
  'no_answer_probability': 0.883711576461792},
 {'id': 'dea7174409afbfe0af0ace21e7f318ae_9',
  'prediction_text': 'we mail you an insurance inquiry letter to allow you to clear up the problem. We send 500,000 inquiry letters a year. ',
  'no_answer_probability': 0.0004379140445962548}]

## Simple model based on a cosine similarity

### Build new datasets (much more simple ones)

We know our ouputs and their ground thruths, we must now build our inputs. 

In [14]:
train_dialogues = load_doc2dial_dataset(name="dialogue_domain", split="train")
val_dialogues = load_doc2dial_dataset(name="dialogue_domain", split="validation")

documents = load_doc2dial_dataset(name="document_domain", split="train")

train_data = load_doc2dial_dataset(name="doc2dial_rc", split="train")
val_data = load_doc2dial_dataset(name="doc2dial_rc", split="validation")

Reusing dataset doc2dial (./data_cache/doc2dial/dialogue_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee)
Reusing dataset doc2dial (./data_cache/doc2dial/dialogue_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee)
Reusing dataset doc2dial (./data_cache/doc2dial/document_domain/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee)
Reusing dataset doc2dial (./data_cache/doc2dial/doc2dial_rc/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee)
Reusing dataset doc2dial (./data_cache/doc2dial/doc2dial_rc/1.0.1/cf6d3ed4e77cea477387dd51c171a021a09bd314cf3a2cb2a6431ca738c6c0ee)


In [15]:
train_data[0]

{'answers': {'answer_start': [346],
  'text': ['you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. ']},
 'context': 'Many DMV customers make easily avoidable mistakes that cause them significant problems, including encounters with law enforcement and impounded vehicles. Because we see customers make these mistakes over and over again , we are issuing this list of the top five DMV mistakes and how to avoid them. \n\n1. Forgetting to Update Address \nBy statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. It is not sufficient to only: write your new address on the back of your old license; tell the United States Postal Service; or inform the poli

In [27]:
def add_spans(sample, dialogues):
    dial_id, turn_id = sample['id'].split('_')

    dials = dialogues.filter(lambda ex: ex['dial_id'] == dial_id)[0]
    print('DIAL: ',dial)
    doc_id = dial['doc_id']   ## corresponding document id
    span_ids = []             ## corresponding span ids

    for tr in dial['turns']:
        if tr['turn_id'] == int(turn_id)+1:

            for sp in tr['references']:
                span_ids.append(sp['sp_id'])

            break

    doc = documents.filter(lambda ex: ex['doc_id'] == doc_id)[0]

    spans = {}
    answer_spans = {}
    for span in doc['spans']:
        spans[span['id_sp']] = span['text_sp']
        if span['id_sp'] in span_ids:
            answer_spans[span['id_sp']] = span['text_sp']

    sample['spans'] = spans
    sample['answers']['spans'] = answer_spans
    return sample

In [28]:
# new_data_train = data_train.map(add_spans)

# new_data_train[0]

In [29]:
# make train data
# %%capture
from tqdm import tqdm
import os

data_file = 'doc2dial_rc_train.csv'
if os.path.exists(data_file):
    new_train_data = pd.read_csv(data_file)
else:
    new_train_data = pd.DataFrame(columns=['id', 'question', 'context', 'answers', 'spans',  'domain', 'title'])
    for ex in tqdm(train_data):
      new_train_data = new_train_data.append(add_spans(ex, train_dialogues), ignore_index=True)
    new_train_data.head()
    new_train_data.to_csv(data_file)

new_train_data.head()

  0%|                                                               | 0/20598 [00:00<?, ?it/s]

DIAL:  dea7174409afbfe0af0ace21e7f318ae


TypeError: string indices must be integers

In [ ]:
new_train_data

In [ ]:
# make val data
# %%capture
data_file = 'doc2dial_rc_val.csv'
if os.path.exists(data_file):
    new_val_data = pd.read_csv(data_file)
else:
    new_val_data = pd.DataFrame(columns=['id', 'question', 'context', 'answers', 'spans',  'domain', 'title'])
    for ex in tqdm(val_data):
      new_val_data = new_val_data.append(add_spans(ex, val_dialogues), ignore_index=True)
    new_val_data.to_csv(data_file)

new_val_data.head()

### Test how to use a doc2vec model 

In [ ]:
# import ast
# new_val = new_val_data.apply(lambda row : ast.literal_eval(row), axis=1)

# new_val.head()

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
import ast

index = 0
instance = new_train_data.iloc[index]
# print((instance['spans']))
docs = ast.literal_eval(instance['spans'])
print(docs)

# Tokenization of each document
tokenized_docs = {}
for i, d in docs.items():
    tokenized_docs[int(i)] = word_tokenize(d.lower())
print(tokenized_docs)

# Convert tokenized document into gensim formated tagged data
tagged_data = []
for i, d in tokenized_docs.items():
  tagged_data.append(TaggedDocument(d, [i]))

tagged_data

In [ ]:
## Train doc2vec model
model = Doc2Vec(tagged_data, vector_size=20, window=2, min_count=1, workers=4, epochs = 100)

## Print model vocabulary
# model.wv.vocab

In [ ]:
# find most similar doc 
# test_doc = ast.literal_eval(instance['answers'])['text'][0]
test_doc = instance['question'][5:]
print(test_doc)
print()
print(ast.literal_eval(instance['answers'])['spans'].keys())
predictions  = model.docvecs.most_similar(positive=[model.infer_vector(test_doc)], topn=30)
predictions

In [ ]:
## Generate predictions in the right format

text = ''
count = 0
sum_score = 0
pred = 0
while True:

  id, score = predictions[pred]
  text += docs[str(id)]
  count += 1
  sum_score += score

  pred += 1
  next_id, next_score = predictions[pred]
  if next_id != id+1:
    break

final_score = sum_score / count

print('Question:', instance['question'][5:])
print('--------------------------------\n')
print('Correct answer:   ', ast.literal_eval(instance['answers'])['text'][0])
print()
print('Our answer:       ', text)

In [ ]:
print('Anwswer in required format:\n')
final_prediction = {"id":instance["id"], "prediction_text":text, "no_answer_probability":1.0-final_score}
final_prediction

### Wrap everything and make predictions on the val set

In [ ]:
def train_predict_doc2vec(dataset, index):
  instance = dataset.iloc[index]
  docs = ast.literal_eval(instance['spans'])

  tokenized_docs = {}
  for i, d in docs.items():
      tokenized_docs[int(i)] = word_tokenize(d.lower())
      # if int(i)==0:
      #   print("gotcha")

  tagged_data = []
  for i, d in tokenized_docs.items():
    tagged_data.append(TaggedDocument(d, [i]))
    # if i==0:
    #   print("gotcha")

  model = Doc2Vec(tagged_data, vector_size=20, window=2, min_count=1, workers=4, epochs = 100)

  test_doc = instance['question'][5:]
  predictions = model.docvecs.most_similar(positive=[model.infer_vector(test_doc)], topn=30)
  # print(predictions)

  text = ''
  count = 0
  sum_score = 0
  pred = 0
  while True:

    id, score = predictions[pred]
    text += docs.get(str(id), '')
    count += 1
    sum_score += score

    pred += 1
    next_id, next_score = predictions[pred]
    if next_id != id+1:
      break

  final_score = sum_score / count
  return {"id":instance["id"], "prediction_text":text, "no_answer_probability":1.0-final_score}

In [ ]:
from tqdm import tqdm

preds = []
new_val_data_slice = new_val_data[:100]

for i in tqdm(range(len(new_val_data_slice))):
  preds.append(train_predict_doc2vec(new_val_data_slice, i))

preds

In [ ]:
import json

file = 'predictions_subtask1_cosine.json'
with open(file, 'w') as outfile:
    json.dump(all_preds, outfile)

In [ ]:
import os
cmd = 'python sharedtask_utils.py --task subtask1 --prediction_json '+file
os.system(cmd)